#Setting

In [1]:
!pip install transformers==4.17 datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from datasets import load_dataset

dataset = load_dataset('klue', 'wos')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'domains', 'dialogue'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['guid', 'domains', 'dialogue'],
        num_rows: 1000
    })
})

# Fine-tune a pretrained model

## Prepare a dataset

In [3]:
dst_feature = dataset["train"].features['domains']
print(dst_feature)

[Value(dtype='string', id=None)]


In [4]:
dataset["train"][0]

{'guid': 'wos-v1_train_00000',
 'domains': ['관광', '식당'],
 'dialogue': [{'role': 'user',
   'text': '서울 중앙에 있는 박물관을 찾아주세요',
   'state': ['관광-종류-박물관', '관광-지역-서울 중앙']},
  {'role': 'sys',
   'text': '안녕하세요. 문화역서울 284은 어떠신가요? 평점도 4점으로 방문객들에게 좋은 평가를 받고 있습니다.',
   'state': []},
  {'role': 'user',
   'text': '좋네요 거기 평점은 말해주셨구 전화번호가 어떻게되나요?',
   'state': ['관광-종류-박물관', '관광-지역-서울 중앙', '관광-이름-문화역서울 284']},
  {'role': 'sys', 'text': '전화번호는 983880764입니다. 더 필요하신 게 있으실까요?', 'state': []},
  {'role': 'user',
   'text': '네 관광지와 같은 지역의 한식당을 가고싶은데요 야외석이 있어야되요',
   'state': ['관광-종류-박물관',
    '관광-지역-서울 중앙',
    '관광-이름-문화역서울 284',
    '식당-지역-서울 중앙',
    '식당-종류-한식당',
    '식당-야외석 유무-yes']},
  {'role': 'sys', 'text': '생각하고 계신 가격대가 있으신가요?', 'state': []},
  {'role': 'user',
   'text': '음.. 저렴한 가격대에 있나요?',
   'state': ['관광-종류-박물관',
    '관광-지역-서울 중앙',
    '관광-이름-문화역서울 284',
    '식당-가격대-저렴',
    '식당-지역-서울 중앙',
    '식당-종류-한식당',
    '식당-야외석 유무-yes']},
  {'role': 'sys', 'text': '죄송하지만 저렴한 가격대에는 없으시네요.', 'state': []},
  

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
tokenizer.is_fast

True

In [7]:
dataset["train"][0]["dialogue"]

[{'role': 'user',
  'text': '서울 중앙에 있는 박물관을 찾아주세요',
  'state': ['관광-종류-박물관', '관광-지역-서울 중앙']},
 {'role': 'sys',
  'text': '안녕하세요. 문화역서울 284은 어떠신가요? 평점도 4점으로 방문객들에게 좋은 평가를 받고 있습니다.',
  'state': []},
 {'role': 'user',
  'text': '좋네요 거기 평점은 말해주셨구 전화번호가 어떻게되나요?',
  'state': ['관광-종류-박물관', '관광-지역-서울 중앙', '관광-이름-문화역서울 284']},
 {'role': 'sys', 'text': '전화번호는 983880764입니다. 더 필요하신 게 있으실까요?', 'state': []},
 {'role': 'user',
  'text': '네 관광지와 같은 지역의 한식당을 가고싶은데요 야외석이 있어야되요',
  'state': ['관광-종류-박물관',
   '관광-지역-서울 중앙',
   '관광-이름-문화역서울 284',
   '식당-지역-서울 중앙',
   '식당-종류-한식당',
   '식당-야외석 유무-yes']},
 {'role': 'sys', 'text': '생각하고 계신 가격대가 있으신가요?', 'state': []},
 {'role': 'user',
  'text': '음.. 저렴한 가격대에 있나요?',
  'state': ['관광-종류-박물관',
   '관광-지역-서울 중앙',
   '관광-이름-문화역서울 284',
   '식당-가격대-저렴',
   '식당-지역-서울 중앙',
   '식당-종류-한식당',
   '식당-야외석 유무-yes']},
 {'role': 'sys', 'text': '죄송하지만 저렴한 가격대에는 없으시네요.', 'state': []},
 {'role': 'user',
  'text': '그럼 비싼 가격대로 다시 찾아주세요',
  'state': ['관광-종류-박물관',
   '관광-지역-서울 중앙',
   '관광

In [9]:
# inputs = tokenizer(dataset["train"][0], is_split_into_words=False)
# print(inputs.tokens())

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).